# Compute conformer energies for a small molecule

This notebook illustrates reading conformers of a molecule from an SDF file and computation of vacuum conformer energies using a SMIRNOFF force field.

In [2]:
# First, generate an example SDF file that contains multiple conformations of a molecule.
# We illustrate how to do this using the Open Force Field toolkit, but you can use your own input files.
from openforcefield.topology import Molecule
#smiles = 'CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(=N3)C)N4CCN(CC4)CCO'
smiles = 'C1CCC(C1)[C@@H](CC#N)N2C=C(C=N2)C3=C4C=CNC4=NC=N3'
molecule = Molecule.from_smiles(smiles)
conf_mol = Molecule.from_smiles(smiles)
molecule.name = 'ruxolitinib'
conf_mol.generate_conformers(n_conformers=10)
for conf_idx in range(10):
    molecule._conformers = None
    molecule.add_conformer(conf_mol.conformers[conf_idx])
    molecule.to_file(f'temp{conf_idx}.sdf', file_format='SDF')
                        

In [3]:
! rm dasatinib_conformers.sdf
! cat temp*.sdf >> ruxolitinib_conformers.sdf

In [4]:
# Load in the molecule and its conformers.
# You would start here if you had your own SDF input files.
# Note that all conformers of the same molecule are loaded as separate Molecule objects
from openforcefield.topology import Molecule
#loaded_molecules = Molecule.from_file('imatinib_conformers.sdf')
loaded_molecules = Molecule.from_file('ruxolitinib_conformers.sdf')
# Collatate all conformers of the same molecule
# NOTE: This isn't necessary if you have already loaded or created multi-conformer molecules;
# it is just needed because our SDF reader does not automatically collapse conformers.
molecules = [loaded_molecules[0]]
for molecule in loaded_molecules[1:]:
    if molecule == molecules[-1]:
        for conformer in molecule.conformers:
            molecules[-1].add_conformer(conformer)
    else:
        molecules.append(molecule)

n_molecules = len(molecules)
n_conformers = sum([mol.n_conformers for mol in molecules])
print(f'{n_molecules} unique molecule(s) loaded, with {n_conformers} total conformers')

1 unique molecule(s) loaded, with 10 total conformers


In [5]:
# Load the openff-1.0.0 force field appropriate for vacuum calculations (without constraints)
from openforcefield.typing.engines.smirnoff import ForceField
forcefield = ForceField('openff_unconstrained-1.0.0.offxml')

In [6]:
# Loop over molecules and compute energies of each conformer
for molecule in molecules:
    print('%s : %d conformers' % (molecule.name, molecule.n_conformers))
    # Create an OpenMM System for the small molecule in vacuum
    system = forcefield.create_openmm_system(molecule.to_topology())
    # Compute energy for all conformers
    from simtk import openmm, unit
    integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName('Reference')
    context = openmm.Context(system, integrator, platform)
    for conformer_index, conformer in enumerate(molecule.conformers):
        context.setPositions(conformer)
        potential = context.getState(getEnergy=True).getPotentialEnergy()
        print('Conformer %5d / %5d : %8.3f kcal/mol' % (conformer_index, molecule.n_conformers, potential/unit.kilocalories_per_mole))
    # Clean up OpenMM Context
    del context, integrator

ruxolitinib : 10 conformers


Conformer     0 /    10 :   64.305 kcal/mol
Conformer     1 /    10 :   74.356 kcal/mol
Conformer     2 /    10 :  110.403 kcal/mol
Conformer     3 /    10 :   70.649 kcal/mol
Conformer     4 /    10 :   72.200 kcal/mol
Conformer     5 /    10 :   73.798 kcal/mol
Conformer     6 /    10 :   72.723 kcal/mol
Conformer     7 /    10 :   62.585 kcal/mol
Conformer     8 /    10 :   68.459 kcal/mol
Conformer     9 /    10 :   70.575 kcal/mol
